# Problem Statement

1. Who is not eligible for benefit? (outlier)
2. How much we should pay for eligible families?

In [1]:
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# Dataset Description
-


# Load The Dataset

In [16]:
def read_files():
    folder = glob.glob("./data/*.csv")
    dfs = [pd.read_csv(f, sep=",", header=0) for f in folder]

    drop = {'drop_application': ['START_TIME', 'END_TIME', 'BIRTH_DATE', 'FIRST_NAME', 'LAST_NAME', 'EMAIL', 'HOME_AREA_CODE', 'PHONE_NUMBER', 'EMPLOYMENT_STATUS'],
            'drop_dependent': ['P8N', 'PARTNER_P8N']}
    dfs[0] = dfs[0].drop(drop['drop_application'], axis=1)
    dfs[2] = dfs[2].drop(drop['drop_dependent'], axis=1)

    # don't need to merge server-log-data and spouse-common-law-data
    df = dfs[0].merge(dfs[1], on='APP_ID').merge(dfs[2], on='APP_ID').merge(dfs[3], on='APP_ID')

    return df

In [17]:
file = read_files()

print(file.shape)
file.head()

C:\Users\naerkond\AppData\Local\Temp\ipykernel_12680\3651096126.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f, sep=",", header=0) for f in folder]


(186958, 22)


,APP_ID,P8N,PARTNER_P8N,STREET_NAME,MUNICIPALITY_NAME,POSTAL_CODE,PROVINCE__ID,CUMUL_PAY,FINANCIAL_INSTITUTION_NUMBER,TRANSIT_NUMBER,...,DATE_BIRTH_2,DATE_BIRTH_3,DATE_BIRTH_4,DATE_BIRTH_5,DATE_BIRTH_6,DATE_BIRTH_7,DATE_BIRTH_8,DATE_BIRTH_9,employment_income_1,employment_income_2
0,100000,39916021.0,65771788.0,8191 Blankenship Gateway,COURTICE,L1E0J2,6.0,800,925,6.709730e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1176.136251,416.158132
1,100001,28596991.0,25829667.0,4163 Werner Meadow Suite 321,SURREY,V3W9W5,10.0,1500,284,7.343455e+11,...,2022-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21058.120853,13290.879147
2,100002,65751811.0,63839036.0,849 Timothy Vista,VANCOUVER,V5Z1Y2,10.0,1500,248,1.454532e+11,...,2013-06-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7423.732510,986.966806
3,100003,43012353.0,73277757.0,8492 Odonnell Squares,MARKHAM,L3R8A5,6.0,800,499,7.194849e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31129.608140,0.000000
4,100004,34120896.0,72359770.0,625 Arthur Forest Suite 533,RICHMOND,V7A3Y7,10.0,800,248,2.990500e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12181.918095,1363.765599


# Data Cleaning

In [18]:
file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186958 entries, 0 to 186957
Data columns (total 22 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   APP_ID                        186958 non-null  int64  
 1   P8N                           186921 non-null  float64
 2   PARTNER_P8N                   158996 non-null  float64
 3   STREET_NAME                   186958 non-null  object 
 4   MUNICIPALITY_NAME             186958 non-null  object 
 5   POSTAL_CODE                   186844 non-null  object 
 6   PROVINCE__ID                  186844 non-null  float64
 7   CUMUL_PAY                     186958 non-null  int64  
 8   FINANCIAL_INSTITUTION_NUMBER  186958 non-null  int64  
 9   TRANSIT_NUMBER                186958 non-null  float64
 10  ACCOUNT_NUMBER                186958 non-null  float64
 11  DATE_BIRTH_1                  186958 non-null  object 
 12  DATE_BIRTH_2                  38830 non-null

In [19]:
# check for null
file.isnull().sum()

APP_ID                               0
P8N                                 37
PARTNER_P8N                      27962
STREET_NAME                          0
MUNICIPALITY_NAME                    0
POSTAL_CODE                        114
PROVINCE__ID                       114
CUMUL_PAY                            0
FINANCIAL_INSTITUTION_NUMBER         0
TRANSIT_NUMBER                       0
ACCOUNT_NUMBER                       0
DATE_BIRTH_1                         0
DATE_BIRTH_2                    148128
DATE_BIRTH_3                    178752
DATE_BIRTH_4                    184491
DATE_BIRTH_5                    186435
DATE_BIRTH_6                    186955
DATE_BIRTH_7                    186958
DATE_BIRTH_8                    186958
DATE_BIRTH_9                    186958
employment_income_1                  0
employment_income_2                  0
dtype: int64

In [ ]:
# draft: make all null = 0 